### Distribution Transformation

* Distribution transformation is a very useful tool which is used to transform the PDF of a generic random variables to a uniform distribution and vice versa.
    * It will be very useful in implementing the copula.
    * Sometimes this technique is called *probability integral transform* or (*percentile-to-percentile transform*). 

* Computationally this method involves computing the quantile function of the distribution, (i.e. computing the inverse of the cumulative distribution function (CDF) of the distribution);
    * here the uniform samples have to be interpreted as cumulative probabilities...

#### Example

* To convert uniformly distributed samples to standard normal it is enough to apply the inverse of CDF. 

* Generically the transformation between uniform and a generic distribution can be summarized as follows (in $\tt{python}$ pseudocode)

$$\textrm{uniform_sample} \rightarrow \tt{distribution.ppf(uniform\_sample)} \rightarrow \textrm{distribution_sample}$$
$$\textrm{distribution_sample} \rightarrow \tt{distribution.cdf(distribution\_sample)} \rightarrow \textrm{uniform_sample}$$

In [ ]:
# make table uniform to gauss
from scipy.stats import norm


* The same transformation may be applied directly on the entire sample ($\tt{rvs}$ samples $\tt{size}$ times from the specified distribution).

In [ ]:
# sample from uniform
from scipy.stats import uniform


* Next apply $\tt{ppf()}$ to the list of $x$ directly.

In [ ]:
# transform to gaus


<img src="uniform_gauss.png">

* Plot them togheter in a 2D graph we can get a sense of what is going on when using the inverse CDF transformation:
    * the inverse CDF stretches the outer regions of the uniform to yield a normal distribution. 

<img src="uniform_to_gauss_2d.png">

* This technique can be used with any arbitrary (univariate) probability distributions, like for example t-Student or Gumbel.

In [1]:
# do the same with t-student
from scipy.stats import t


<img src="uniform_tstudent_2d.png">

* To go from any arbitray distribution to the uniform we can just apply the inverse of the inverse CDF, which is the CDF itself...

In [ ]:
# make uniform to gauss to uniform


<img src="full_chain.png">

## Copula

* In probability theory a *copula* $\mathcal{C}(F_1, F_2, \ldots, F_n)$ is a multivariate (multidimensional) cumulative distribution function whose marginal probability distributions (the probability distribution of each dimension) are uniform on the interval $[0,1)$. 


* Copulas are popular since allow to easily model the distribution of correlated random variables by representing marginals and correlation separately. 
    * Essentially a complicated problem can be split into many simpler components, so the copula results to be very handy.
    * Copulas are widely used in quantitative finance to model risk. 


#### Example

* Imagine two correlated variables: look at various rivers and for each one record its maximum water level, and also count how many months each river caused flooding. 
    * The maximum level of the river is assumed to be distributed as a Gumbel;
    * the number of floods can be modelled according to a Beta distribution.


* It is reasonable to assume that the two quantities are correlated, however we don't know how we could model that correlated probability distribution. 
    * Before we only specified the distributions for individual variables, irrespective of the other one (i.e. the marginals);
    * we would like to study the joint distribution of both of these together. 


* Copulas allow to decompose a joint probability distribution into the marginals (which by definition have no correlation) and a function which couples (hence the name) them together.

## Adding Correlation with Gaussian Copulas

* The next steps are quite simple in light of what shown at the beginning:
    * indeed we know how to convert every PDF from and to a uniform distribution; 
    * we can generate uniformly distributed data with the desired correlation;
    * then transform those uniform marginals into the real PDF. 


* For simplicity we ara going to assume Gaussian correlation, but the same holds for other kind of correlation. 
    1. Simulate from a multivariate Gaussian with the specific corrrelation structure;
    2. transform each Gaussian marginal to uniform;
    3. finally transform the uniforms to whatever we like.

In [ ]:
# sample from multi-normal with corr
from scipy.stats import multivariate_normal


<img src="multivariate_2d.png">

* Tranform the marginals to uniform using the $\tt{cdf}$ function of the normal distribution/
    * $x$ is a 2D vector in this case, but $\tt{scipy}$ methods are such that the tranformation will be applied separately on each component.

In [ ]:
# convert each component
from scipy.stats import norm


<table>
    <tr>
        <td><img src="copula_2d.png" width=300></td>
        <td><img src="copula_3d.png" width=300></td>
    </tr>
</table>

* **Since we used a multivariate stadard normal to model correlation this is also called a Gaussian Copula.**


* Finally we can transform the marginals again from uniform to what we want (i.e. Gumbel and Beta in the river example). 

In [ ]:
# convert again to gumbel e beta
from scipy.stats import gumbel_l, beta


* This particular $\tt{python}$ notation is called *slicing* of an array and it is very powerful. 


* Imagine to have a 2D matrix (a 2D array in $\tt{python}$ terminology):
    * to access an element of the matrix it is necessary to specify two indices (the row and the column of the element);
    * to access a range of values it is possible to specify the interval of indices inside the square brackets;
    * to access the entire colum (or row) it is enough to write $\tt{[:]}$.

In [ ]:
# examples of slicing 
import numpy as np


* Compare the scatter plot of Gumbel and Beta distribution with correlation and without.

In [ ]:
# sample from Gumbel
# sample from Beta

<table>
    <tr>
        <td><img src="gumbel_beta_corr.png" width=300></td>
        <td><img src="gumbel_beta_uncorr.png" width=300></td>
    </tr>
</table>

* Using the uniform distribution as a common base for our transformations we can easily introduce correlations and flexibly construct complex probability distributions. 
    * Everything shown is directly extendeable to higher dimensional distributions as well.

## Modelling Correlation between Risks
In credit derivative valuation and credit risk management, one of the most important issue is the estimate of default probabilities and their correlations. 

Default correlation measures the tendency of two companies to default at about the same time. For this, generally speaking, there are two ways: using historical default data or using mathematical models, like copulas. 

Historical default data has played an important role in the estimation of default probabilities. However, because default events are rare, there is very limited default data available. Moreover, historical data reflects the historical default pattern only and it may not be a proper indicator of the future. This makes the estimation of default probabilities from historical data difficult and inexact. To use this same data to estimate default correlations is even more difficult and more inexact. 

On the other hand mathematical models don't rely on historical default data. So this is the way that is now used to estimate/apply correlations.

### Correlated Probability of Default

Let's now see how copulas can be used to sample numbers from correlated distributions. These are the steps to follow:

* generate a random vector $\mathbf{x}=(x_1, x_2,\ldots)$ from a multivariate distribution with the desired correlation;
* determine every $U_i(x_i)$ by applying $\tt{cdf}$ to each $x_i$;
* transform again each $U_i(x_i)$ to the desired marginal distributions using $\tt{ppf}$.

Each component of the vector $\mathbf{x}$ is now transformed as it was drawn from the desired marginals with the appropriate correlation.


* Imagine three companies (A, B and C) which have a cumulative probability of defaulting within the next two years of 10%
    * compute the probabilities to have the three of them all defaulting within the next two years in the two cases: 
        * with independent default probabilities;
        * and correlated default probabilities.


* Independent probabilities: the odds to get three defaults within two years is the product of the single probabilities:

$$\mathbb{P}_{\mathrm{uncorr}}= 10\%\cdot 10\%\cdot 10\% = 0.1\%$$

* To verify it in $\tt{python}$ just implement a Monte Carlo simulation which sample from the marginal distribution (obtained from the copula function) and count how many times the three default probabilites are lower than 10%. 


* The final probability will be the ratio of the numer of successes by the total trials.
    * Since we are just interested in the cumulative probabilities, it is sufficent to work with the uniforms.

In [ ]:
# test default probabilities of three companies (no corr)
from scipy.stats import multivariate_normal, norm


* For the second case it is enough to repeat the same Monte Carlo experiment this time starting with perfectly correlated default probabilities.

In [ ]:
# test default probabilities of three companies (perfect corr)
from scipy.stats import multivariate_normal, norm


* In this case the result is close to 10%, like we had only one single company, indeed being perfectly correlated either there is no default or three "simultaneous" defaults with 10% probability.

## Gaussian Copula Model

While there are several types of copula function models, the first introduced was the *one-factor Gaussian copula model*. This model has, above all, the advantage that can be solved analytically.

Consider a portfolio of $N$ bonds and assume that the marginal probabilities of default are known for each issuer. Define:

* $t_i$, the time of default of the $i^{th}$ company:
* $Q_i(t)$, the cumulative probability that company $i$ will default before time $t$; that is, the probability that $t_i \le t$.

To generate a one-factor model for the $t_i$ we define random
variables $X_i$ $(1\le i \le N)$

$$X_i = a_i M + \sqrt{1-a_i^2}Z_i,\qquad i = 1, 2,\ldots, N$$

where $M$ and the $Z_i$ are independent zero-mean unit-variance  distributions (hence $X_i$ are also distributed with zero-mean and unit standard-deviation) and $-1 \le a_i \lt 1$.

The previous equation defines a correlation structure between the
$X_i$ which are dependent on a single common factor $M$. The $Z_i$ term is usually 
called the idiosyncratic component of default. 
The correlation between $X_i$ and $X_j$ is

$$\mathrm{Corr}(X_i, X_j) = \cfrac{\mathbb{E}[(X_i-\mu_i)(X_j-\mu_j)]}{\sigma_{i}\sigma_{j}} =\mathbb{E}[X_i X_j] = a_i a_j \mathbb{E}[M^2] = a_i a_j
$$
where we just exploit the definition of $X_i$ and its properties.

If $F_i$ is the cumulative distribution function of the $X_i$,
with a percentile to percentile transformation we can map the $X_i$ to the $t_i$, so that $Q_i(t_i) = \mathbb{P}(X_i\le x)=F_i(x)$.
Therefore the point $X_i = x$ is transformed to $t_i = t$ where
$x = F_i^{-1}[Q_i(t)]$.

Let's note that, *conditional* on $M$, the $N$ default events are independent. So we can write

$$
\begin{split}
Q_i(t_i|M) = \mathbb{P}(X_i\le x|M) &= \mathbb{P}(a_i M + \sqrt{1-a_i^2}Z_i\le x|M) =\\
&= \mathbb{P}\left(Z_i\le \cfrac{x-a_i M}{\sqrt{1-a_i^2}}\right)
=H_i\left(\cfrac{F^{-1}[Q_i(t_i)]-a_i M}{\sqrt{1-a_i^2}}\right)
\end{split}
$$
where $H_i$ is the cumulative distribution function of the $Z_i$.

Although in principle any distribution could be used for $M$ and the
$Z$'s (provided they have zero mean and unit variance), one common
choice is to let them be standard normal distributions (resulting in a
Gaussian copula).
So we can specialize the previous equation as

$$
Q_i(t_i|M) = \Phi\left(\cfrac{\Phi^{-1}[Q_i(t_i)]-a_i M}{\sqrt{1-a_i^2}}\right)
$$
where $\Phi$ denotes the cumulative distribution function of the standard normal distribution.

If we call $\mathcal{C}(t_1,\ldots,t_N)$ the joint distribution of the default times of the $N$ bonds  in the portfolio then

$$
\mathcal{C}(t_1,\ldots,t_N)=\Phi_{A}(\Phi^{-1}(Q_1(t_1)),\ldots,\Phi^{-1}(Q_N(t_N)))
$$
where $A$ is the correlation matrix of the default probabilities, is the one factor Gaussian copula model (one factor because there is only a random variable, $M$, which determines the correlation between $X_i$).

Clearly different choices of distributions result in different copula models, and in different natures of the default dependence. For example, copulas where the \(M\) have heavy tails generate models where there is a
greater likelihood of a clustering of early defaults for several
companies.

### Standard Market Model

Assume the following two assumptions are made:

* all the companies have the same default intensity (hazard rates), i.e, $\lambda_i = \lambda$ (which means they all have the same default probabilities);
* the pairwise default correlations are the same, i.e $a_i = a$; in other words the contribution of the market 	component $M$ is the same for all the companies and the correlation between any two companies is constant, $\rho = a^2$.

Under these assumptions, given the market situation $M = m$, all the
companies have the same cumulative default probability
$Q_i(t_i|m)=\mathbb{P}(X_i \le x|m)$. 
Moreover, for a given value of the
market component $M$, the defaults are mutually independent for all
the underlying companies. 

## Binomial Distribution

In statistics, the binomial distribution is a discrete probability of independent events, where each event has exactly two possible outcomes. For example, if we toss a coin 10 times and we are interested in how many times it will land on heads, the probability of it lending one, two, three, up to 10 head in a row will form a distribution of probabilities or binomial distribution.

The binomial distribution answers to the question which is the probability of exactly k successes on n repeated trials given that:
 
* the number of observations or trials is fixed;
* each observation or trial is independent. Like in the coin toss, the previous toss doesn’t affect the following;
* the probability of success is exactly the same from one trial to another. Each coin toss has 50% chances to fall in heads or tails.

$$\mathrm{PMF_{binomial}} = \binom{n}{k} p^{k}(1-p)^{n-k}$$
where $\binom{n}{k}$ is the binomial coefficient and is equal to $\frac{n!}{k!(n−k)!}$, $p$ is the probability of an outcome.

In [ ]:
# probability of tossing a coin 
from scipy.stats import binom
print (binom(10, 0.5).pmf(5))

In [ ]:
from matplotlib import pyplot as plt

x = range(20)
plt.plot(x, binom(10, 0.5).pmf(x))
plt.grid(True)
plt.show()

In [ ]:
b = binom(10, 0.5)

s = 0
for i in range(5, 11):
    s += b.pmf(i)
print (s)

In [ ]:
print (b.cdf(10)-b.cdf(4))
print (1 - b.cdf(4))

Letting $l_{t|m}$ be the total defaults that
have occurred by time $t$ conditional on the market condition
$M = m$, then $l_{t|m}$ follows a binomial distribution and

$$Q(l_{t|m} = j) = \cfrac{N!}{j!(N-j)!}Q^j_{t|m}(1-Q_{t|m})^{N-j},\qquad  j=0, 1, 2,\ldots,N$$

In general to evaluate any function of $Q$, $g(Q(l_t))$, regardless the parameter $M$ is necessary to average according to the following integral

$$
g(Q(l_{t} = j)) = \int_{-\infty}^{\infty}{g(Q(l_{t|m} = j))\cdot f_M(m)dm}
$$

where $f_M(m)$ is the probability density function (PDF) of the random
variable $M$ (which with our assumption is a standard normal).

With the assumption that have been made the one factor model is also called *Market Standard Model*.

### Extensions of the One Factor Copula Model
Many other one-factor model have been tried: Student t copula, Clayton copula and many others. In general we can define a new model by simply choosing particular functions for $M$ and $Z_i$ provided they are with mean zero and standard deviation 1. 

If instead of the single factor $M$ there are two or more

$$
X_i = a_1 M_1 + a_2 M_2 + \sqrt{1 - a_1^2 - a^2_2}Z_i
$$
and similarly
$$
Q(t|M_1, M_2) = \Phi\left(\cfrac{\Phi^{-1}[Q(t)]-a_1 M_1 - a_2 M_2}{\sqrt{1 - a_1^2 - a^2_2}}\right)
$$

This kind of models are proportionally slower with the increase of the number of factors.
Later we will also see that the correlation can be determined directly from the market quotes.

## Basket Default Swaps
A basket default swap is a credit derivative on a portfolio of reference
entities. The simplest basket default swaps are first-to-default,
second-to-default, or nth-to-default swaps. 

This kind of contracts are very similar to normal CDS except for the protection they offer.
With respect to a basket of reference entities, a first-to-default swap provides insurance for only the first default, a second-to-default swap provides insurance
for only the second default, and a nth-to-default swap provides insurance for only the $n^{th}$ default. 

For example, in the last case, the
seller does not make a payment to the protection buyer for
the first $n-1$ defaulted reference entities, and makes a payment only for the
$n^{th}$ defaulted reference entity. Once there has been this payment the swap terminates.


### Basket CDS Valuation under Market Standard Model
We now present some numerical results for an $n$th-to-default basket. We assume that the
principals and expected recovery rates are the same for all underlying reference assets.
The valuation procedure is similar to that for a regular CDS where there is only one
reference entity.

In a regular CDS indeed its valuation is based on the probability that a default
occured between times $t_1$ and $t_2$. Here instead the valuation will be based on the probability that the
$n^{th}$ default was between times $t_1$ and $t_2$.

We assume the buyer of protection makes quarterly payments at a specified rate
until the $n^{th}$ default occurs or the end of the life of the contract is reached. 

In the event of the $n^{th}$ default occurring, the seller pays $F\cdot(1-R)$. 
The contract can be valued by calculating the expected present value
of payments and the expected present value of payoffs. 

Let's implement now a class that represents a Basket Default Swap. The idea is to reuse the CreditDefaultSwap class that we have already implemented since for example, computing the NPV of a basket can be considered equivalent to compute the NPV of a CDS using the default probabilities derived by the one factor copula model.

### $\tt{args}$ and $\tt{kwargs}$

$\tt{args}$ and $\tt{kwargs}$ represent two useful ways of passing parameters to a function that is called by another function.

In [ ]:
# paramters in global space

a = 1
b = 1
c = 5

def runner(f, x):
    return f(x)*2

def func(x):
    return a*x**2 + b*x + c

print (runner(func, 2))

If I had to run many times runner changeing each time the values of the parameters $\tt{a}$, $\tt{b}$ and $\tt{c}$ I would be in troubles. So I can use $\tt{args}$ with the star operator.

In [ ]:
def runner(f, x, args):
    return f(x, *args)*2

def func(x, a, b, c):
    return a*x**2 + b*x + c

for v in range(3):
    print (runner(func, 2, args=(v, 1, 5)))

The minimization algorithm is a clear example of this. If you think of $\tt{runner}$ as the $\tt{minimize}$ function and $\tt{func}$ as the objective function the analogy is quite close. In our examples we have always used the first way.

$\tt{kwargs}$ works like $\tt{args}$ except that you have to pass a dictionary with as keys the names of the parameters. Also to expand it you have to use the double-star operator.

In [ ]:
def runner(f, x, kwargs):
    return f(x, **kwargs)*2

for v in range(3):
    print (runner(func, 2, kwargs={"a":v, "c":5, "b":1}))

Going back to the BasketDefaultSwaps class we will use this technique. In fact the idea is:

* compute the correlated default probability of each name according to 

$$Q(t|M) = \Phi\left(\cfrac{\Phi^{-1}[Q(t)]-\sqrt{\rho} M}{\sqrt{1-\rho}}\right)$$

* compute the probability to have at least j defaults using the binomial distribution

$$Q(l_{t|m} \ge j) = \sum_{k=j}^{N}\left[\cfrac{N!}{k!(N-k)!}Q(t|M)^k(1-Q(t|M))^{N-k}\right]$$

* create a **credit curve**, $CC(Q)$,  with those probabilities;
* re-use the $\tt{CreditDefaultSwap}$ class methods to compute NPV and breakeven rate (which depend on a discount curve and a previous credit curce), through integration of

$$ \mathrm{NPV}(DC, CC(Q)) = \int_{-\infty}^{\infty}{\mathrm{NPV}(DC, CC(Q)) f_M(m)dm} $$
and similarly for the breakeven rate.

Imagine you need to integrate a function $f$ in $\tt{python}$ we can use the $\tt{scipy.integrate.quad}$ function which takes in input, the function to integrate, the integration limits and optional arguments of the integrand.

Imagine to integrate

$$f(x) = \int_{-2}^{4}(ax^{3} + b)~dx$$

for $a=3$ and $b=5$. 

In [ ]:
from scipy.integrate import quad

def func(x, a, b):
    return a*x**3 + b

s = quad(func, -2, 4, args=(3, 5))
print (s)

In [ ]:
from finmarkets import CreditCurve, CreditDefaultSwap
from scipy.stats import norm, binom
from numpy import sqrt, exp
from scipy.integrate import quad
import numpy as np

class BasketDefaultSwaps:
    def __init__(self, notional, names, rho, start_date, spread,
                 maturity, tenor=3, recovery=0.4):
        self.names = names
        self.rho = rho
        self.cds = CreditDefaultSwap(notional, start_date, spread, maturity,
                                     tenor, recovery)
        
    def npv(self, Q_dates, Q, dc, n_defaults):
        v = quad(self.one_factor_model, -np.inf, np.inf,
                 args=(self.cds.npv, Q_dates, Q, dc, n_defaults))
        return v[0]
    
    def one_factor_model(self, M, f, Q_dates, Q, dc, j):
        DP = norm.cdf((norm.ppf(Q)-sqrt(self.rho)*M)/(sqrt(1-rho)))
        b = binom(self.names, DP)
        S = 1 - (1-b.cdf(j-1))
        cc = CreditCurve(Q_dates, S)
        return f(dc, cc)*norm.pdf(M)
    
    def breakeven(self, Q_dates, Q, dc, n_defaults):
        v = quad(self.one_factor_model, -np.inf, np.inf,
                 args=(self.cds.breakevenRate, Q_dates, Q, dc, n_defaults))
        return v[0]

Consider first a 5-year 3rd-to-default CDS on a basket of 10 reference entities in the
situation where the copula correlation is 0.3 and the expected recovery rate, $R$, is $40\%$. The term structure of interest rates
is assumed to be flat at 5%. The default probabilities for the 10 entities are generated by
Poisson processes with constant default intensities (hazard rates), $\lambda_i = 0.01$, so that 

$$Q(t) = 1 - e^{-\lambda t} $$

In [ ]:
from finmarkets import DiscountCurve
from datetime import date
from dateutil.relativedelta import relativedelta
import numpy as np

n_cds = 10
rho = 0.3
l = 0.01
observation_date = date.today()

pillar_dates = [observation_date + relativedelta(years=i) for i in range(6)]
dfs = [1/(1+0.05)**i for i in range(6)]
dc = DiscountCurve(observation_date, pillar_dates, dfs)
Q = [1-np.exp(-(l*t)) for t in range(6)]

ndefaults = 3
basket = BasketDefaultSwaps(1, n_cds, rho, observation_date, 0.01, 5)
print(basket.breakeven(pillar_dates, Q, dc, ndefaults))

## Collateralized Debt Obligation

A Collateralized Debt Obligation (CDO) is a credit derivative where the issuer, typically investment banks, gather risky assets and repackage them into discrete classes (*tranches*) based on the level of credit risk assumed by the investor. These tranches of securities become the final investment product.

Tranches are named to reflect their risk profile: senior, mezzanine and subordinated/equity and are delimited by the attachment ($L$) and detachment points ($U$), which represent the percentages of the total principal defining their boundaries. 
For example, a 5-10% tranche has an attachment point of 5% and a detachment point of 10%. 

<!----When the accumulated loss of the reference pool is no more than 5% of the total initial notional of the pool, the tranche will not be affected. However, when the loss has exceeded 5%, any further loss will be deducted from the tranche's notional until the detachment point, 10%, is reached.-->

Each of these tranches has a different level of seniority relative to the others in the sense that a senior tranche has coupon
and principal payment priority over a mezzanine tranche, while a mezzanine tranche has
coupon and principal payment priority over an equity tranche. 

Indeed they receive returns using a set of rules known as *waterfall*. Incomes of the portfolio are first used to provide returns to the most senior tranche, then to the next and so on.
So the senior tranches are generally safest because they have the first claim on the collateral, although they'll offer lower coupon rates.

It is important to note
that a CDO only redistributes the total risk associated with the underlying pool of assets
to the priority ordered tranches. It neither reduces nor increases the total risk associated
with the pool.

There are various kind of CDOs:

* in a **Cash CDO** the reference portfolio consists of corporate bonds owned by the CDO issuer. Cash flows from collateral are used to pay principal and interest to investors. If such cash flows prove inadequate, principal and interest is paid to tranches according to their seniority. 

<img src="cdo_structure.png">

* in a **Synthetic CDO** the underlying reference portfolio is no longer a physical portfolio of bonds or loans, instead it is a *fictitious* portfolio consisting of a number of names each with an associated notional amount. The value of a synthetic CDO usually comes from insurance premiums of credit default swaps paid for by investors. The seller assumes the underlying assets will perform while the investor, on the other hand, assumes the underlying assets will default.